In [1]:
import pytorch_lightning as pl
from omegaconf import OmegaConf, DictConfig
import nemo.collections.asr as nemo_asr
from nemo.core.config import hydra_runner
from nemo.utils import logging
from nemo.utils.exp_manager import exp_manager
import datetime
from pathlib import Path
import torch
from sbb_project import consts
import copy
from pytorch_lightning.loggers import WandbLogger  
import wandb
from ruamel.yaml import YAML
import functools
import copy

[NeMo W 2023-01-02 18:38:49 optimizers:67] Could not import distributed_fused_adam optimizer from Apex
[NeMo W 2023-01-02 18:38:51 nemo_logging:349] /home/user/.local/lib/python3.9/site-packages/torch/jit/annotations.py:309: UserWarning: TorchScript will treat type annotations of Tensor dtype-specific subtypes as if they are normal Tensors. dtype constraints are not enforced in compilation either.
      warnings.warn("TorchScript will treat type annotations of Tensor "
    


In [2]:
model = nemo_asr.models.ASRModel.from_pretrained(model_name='stt_de_conformer_ctc_large',
                                                    map_location=torch.device("cuda:2"))

[NeMo I 2023-01-02 18:38:51 cloud:66] Downloading from: https://api.ngc.nvidia.com/v2/models/nvidia/nemo/stt_de_conformer_ctc_large/versions/1.5.0/files/stt_de_conformer_ctc_large.nemo to /home/user/.cache/torch/NeMo/NeMo_1.14.0/stt_de_conformer_ctc_large/6ccf7bc56339a48fad7e06265e367dfa/stt_de_conformer_ctc_large.nemo
100% [......................................................................] 451672587 / 451672587[NeMo I 2023-01-02 18:40:27 common:912] Instantiating model from pre-trained checkpoint
[NeMo I 2023-01-02 18:40:31 mixins:170] Tokenizer SentencePieceTokenizer initialized with 128 tokens


[NeMo W 2023-01-02 18:40:31 modelPT:142] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /data/train/tarred_audio_manifest.json
    sample_rate: 16000
    batch_size: 8
    shuffle: true
    num_workers: 16
    pin_memory: true
    use_start_end_token: false
    trim_silence: false
    max_duration: 20.0
    min_duration: 0.1
    shuffle_n: 2048
    is_tarred: true
    tarred_audio_filepaths: /data/train/audio__OP_0..1023_CL_.tar
    
[NeMo W 2023-01-02 18:40:31 modelPT:149] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath:
    - /data/voxpopuli_de/dev/voxpopuli_dev_manifest.json
    - /data/voxpopuli_de/test/voxpopuli_

[NeMo I 2023-01-02 18:40:31 features:267] PADDING: 0
[NeMo I 2023-01-02 18:40:37 save_restore_connector:243] Model EncDecCTCModelBPE was successfully restored from /home/user/.cache/torch/NeMo/NeMo_1.14.0/stt_de_conformer_ctc_large/6ccf7bc56339a48fad7e06265e367dfa/stt_de_conformer_ctc_large.nemo.


In [3]:
model.cfg

{'sample_rate': 16000, 'log_prediction': True, 'ctc_reduction': 'mean_batch', 'train_ds': {'manifest_filepath': '/data/train/tarred_audio_manifest.json', 'sample_rate': 16000, 'batch_size': 8, 'shuffle': True, 'num_workers': 16, 'pin_memory': True, 'use_start_end_token': False, 'trim_silence': False, 'max_duration': 20.0, 'min_duration': 0.1, 'shuffle_n': 2048, 'is_tarred': True, 'tarred_audio_filepaths': '/data/train/audio__OP_0..1023_CL_.tar'}, 'validation_ds': {'manifest_filepath': ['/data/voxpopuli_de/dev/voxpopuli_dev_manifest.json', '/data/voxpopuli_de/test/voxpopuli_test_manifest.json', '/data/mcv7.0_de/mcv7.0_dev_manifest_cleaned.json', '/data/mcv7.0_de/mcv7.0_test_manifest_cleaned.json', '/data/mls_de/mls_dev_manifest_cleaned.json', '/data/mls_de/mls_test_manifest_cleaned.json'], 'sample_rate': 16000, 'batch_size': 8, 'shuffle': False, 'num_workers': 16, 'pin_memory': True, 'use_start_end_token': False}, 'test_ds': {'manifest_filepath': ['/data/voxpopuli_de/test/voxpopuli_test

In [4]:
sweep_config = {
  "method": "random",   # Random search
  "metric": {           # We want to minimize `val_loss`
      "name": "val_wer",
      "goal": "minimize"
  },
  "parameters": {
        "lr": {
            # log uniform distribution between exp(min) and exp(max)
            "distribution": "log_uniform",
            "min": -9.21,   # exp(-9.21) = 1e-4
            "max": -2.30    # exp(-4.61) = 1e-2
        },
      "freq_masks": {
            "distribution": "int_uniform",
            "min": 0,  
            "max": 7     
        },
      "time_masks": {
            "distribution": "int_uniform",
            "min": 0,  
            "max": 15     
        },
        "freq_width": {
            "distribution": "uniform",
            "min": 25,  
            "max": 30     
        },
          "time_width": {
                "distribution": "uniform",
                "min": 0.01,  
                "max": 0.07     
            }      
    }
}

In [5]:
train_manifest = [
        str(consts.MANIFEST_DIR.joinpath(consts.MANIFEST_FILE.format("train", "none"))),
        str(consts.MANIFEST_DIR.joinpath(consts.MANIFEST_FILE.format("train", "neg10"))),
        str(consts.MANIFEST_DIR.joinpath(consts.MANIFEST_FILE.format("train", "0"))),
        str(consts.MANIFEST_DIR.joinpath(consts.MANIFEST_FILE.format("train", "10"))),
        str(consts.MANIFEST_DIR.joinpath(consts.MANIFEST_FILE.format("train", "20"))),
        str(consts.MANIFEST_DIR.joinpath(consts.MANIFEST_FILE.format("train", "30"))),
        str(consts.MANIFEST_DIR.joinpath(consts.MANIFEST_FILE.format("train", "40"))),
    ]
val_manifest = consts.MANIFEST_DIR.joinpath(consts.MANIFEST_FILE.format("val", "random"))
test_manifest = consts.MANIFEST_DIR.joinpath(consts.MANIFEST_FILE.format("test", "random"))

In [6]:
def sweep_iteration():
    
    # set up W&B logger
    wandb.init()    # required to have access to `wandb.config`
    wandb_logger = WandbLogger(log_model='all')  # log final model
        
    trainer = pl.Trainer(max_epochs=10, logger=wandb_logger, devices = [2], accelerator="gpu")
    
    # setup model - note how we refer to sweep parameters with wandb.config
    model = nemo_asr.models.ASRModel.from_pretrained(model_name='stt_de_conformer_ctc_large',
                                                    map_location=torch.device("cuda:2"))

    model.set_trainer(trainer)
    
    model.cfg.train_ds.is_tarred = False
    
    model.cfg.train_ds.manifest_filepath = ','.join(train_manifest)
    model.cfg.validation_ds.manifest_filepath = str(val_manifest)
    model.cfg.test_ds.manifest_filepath = str(test_manifest)
    
    model.cfg.train_ds.max_duration = 45
    model.cfg.train_ds.batch_size = 4
    model.cfg.validation_ds.batch_size = 4
    model.cfg.test_ds.batch_size = 4
    
    model.cfg.optim.lr = wandb.config.lr
    model.cfg.spec_augment.freq_width = wandb.config.freq_width
    model.cfg.spec_augment.freq_masks = wandb.config.freq_masks
    model.cfg.spec_augment.time_width = wandb.config.time_width
    model.cfg.spec_augment.time_masks = wandb.config.time_masks

    model.setup_training_data(model.cfg.train_ds)
    model.setup_validation_data(model.cfg.validation_ds)
    model.setup_test_data(model.cfg.test_ds)
    model.setup_optimization(model.cfg.optim)

    # train
    trainer.fit(model)


In [ ]:
sweep_id = wandb.sweep(sweep_config, project="ASR-Conformer-CTC")
wandb.agent(sweep_id, function=sweep_iteration)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Malformed sweep config detected! This may cause your sweep to behave in unexpected ways.
wandb: WARNING To avoid this, please fix the sweep config schema violations below:
wandb: WARNING   Violation 1. lr uses log_uniform, where min/max specify base-e exponents. Use log_uniform_values to specify limit values.


Create sweep with ID: e8k177b6
Sweep URL: https://wandb.ai/unisg-ds-nlp/ASR-Conformer-CTC/sweeps/e8k177b6


wandb: Agent Starting Run: euod7rij with config:
wandb: 	freq_masks: 4
wandb: 	freq_width: 27.28040152795433
wandb: 	lr: 0.00020775497073580933
wandb: 	time_masks: 7
wandb: 	time_width: 0.05481313238538046
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: marccgrau (unisg-ds-nlp). Use `wandb login --relogin` to force relogin


[NeMo W 2023-01-02 18:40:45 nemo_logging:349] /home/user/.local/lib/python3.9/site-packages/pytorch_lightning/loggers/wandb.py:396: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
      rank_zero_warn(
    
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


[NeMo I 2023-01-02 18:40:45 cloud:56] Found existing object /home/user/.cache/torch/NeMo/NeMo_1.14.0/stt_de_conformer_ctc_large/6ccf7bc56339a48fad7e06265e367dfa/stt_de_conformer_ctc_large.nemo.
[NeMo I 2023-01-02 18:40:45 cloud:62] Re-using file from: /home/user/.cache/torch/NeMo/NeMo_1.14.0/stt_de_conformer_ctc_large/6ccf7bc56339a48fad7e06265e367dfa/stt_de_conformer_ctc_large.nemo
[NeMo I 2023-01-02 18:40:45 common:912] Instantiating model from pre-trained checkpoint
[NeMo I 2023-01-02 18:40:49 mixins:170] Tokenizer SentencePieceTokenizer initialized with 128 tokens


...

[NeMo I 2023-01-02 23:16:16 wer_bpe:260] 
    
[NeMo I 2023-01-02 23:16:16 wer_bpe:261] reference:rangierfahrt vom gleis mike zwei drei ins drei drei antworten
[NeMo I 2023-01-02 23:16:16 wer_bpe:262] predicted:heruf heruff der war kein hunde frau schierfahrt von gleis maik zwei drei ins drei drei antwortete rue immer umerns an sich
[NeMo I 2023-01-02 23:16:16 wer_bpe:260] 
    
[NeMo I 2023-01-02 23:16:16 wer_bpe:261] reference:rangierfahrt vom gleis sieben via zulu acht ins gleis null null antworten
[NeMo I 2023-01-02 23:16:16 wer_bpe:262] predicted:rangierfahrt vom gleis sieben via zulu acht ih sgleis null null antworten
[NeMo I 2023-01-02 23:16:16 wer_bpe:260] 
    
[NeMo I 2023-01-02 23:16:16 wer_bpe:261] reference:spracheingabe
[NeMo I 2023-01-02 23:16:16 wer_bpe:262] predicted:sprach eingabe
[NeMo I 2023-01-02 23:16:16 wer_bpe:260] 
    
[NeMo I 2023-01-02 23:16:16 wer_bpe:261] reference:rangierfahrt von quebec fünf acht ins gleis oscar sieben drei antworten
[NeMo I 2023-01-02 2

Validation: 0it [00:00, ?it/s]

[NeMo I 2023-01-02 23:19:14 wer_bpe:260] 
    
[NeMo I 2023-01-02 23:19:14 wer_bpe:261] reference:rangierfahrt vom gleis mike zwei drei ins drei drei antworten
[NeMo I 2023-01-02 23:19:14 wer_bpe:262] predicted:heruf heruf der war keinhunde frau schierfahrt von gleis maike zwei drei ins drei drei antwortete rue immer umereins an sich
[NeMo I 2023-01-02 23:19:14 wer_bpe:260] 
    
[NeMo I 2023-01-02 23:19:14 wer_bpe:261] reference:rangierfahrt vom gleis sieben via zulu acht ins gleis null null antworten
[NeMo I 2023-01-02 23:19:14 wer_bpe:262] predicted:rangierfahrt vom gleis sieben via zulu acht ih sgleis null null antworten
[NeMo I 2023-01-02 23:19:14 wer_bpe:260] 
    
[NeMo I 2023-01-02 23:19:14 wer_bpe:261] reference:spracheingabe
[NeMo I 2023-01-02 23:19:14 wer_bpe:262] predicted:sprach eingabe
[NeMo I 2023-01-02 23:19:14 wer_bpe:260] 
    
[NeMo I 2023-01-02 23:19:14 wer_bpe:261] reference:rangierfahrt von quebec fünf acht ins gleis oscar sieben drei antworten
[NeMo I 2023-01-02 

Validation: 0it [00:00, ?it/s]

[NeMo I 2023-01-02 23:22:12 wer_bpe:260] 
    
[NeMo I 2023-01-02 23:22:12 wer_bpe:261] reference:rangierfahrt vom gleis mike zwei drei ins drei drei antworten
[NeMo I 2023-01-02 23:22:12 wer_bpe:262] predicted:heruf heruff der war kounde frau schierfahrt von gleis maike zwei drei ins drei drei antwortete rue immer umerns an sich
[NeMo I 2023-01-02 23:22:12 wer_bpe:260] 
    
[NeMo I 2023-01-02 23:22:12 wer_bpe:261] reference:rangierfahrt vom gleis sieben via zulu acht ins gleis null null antworten
[NeMo I 2023-01-02 23:22:12 wer_bpe:262] predicted:rangierfahrt vom gleis sieben via zulu acht ih sgleis null null antworten
[NeMo I 2023-01-02 23:22:12 wer_bpe:260] 
    
[NeMo I 2023-01-02 23:22:12 wer_bpe:261] reference:spracheingabe
[NeMo I 2023-01-02 23:22:12 wer_bpe:262] predicted:sprach eingabe
[NeMo I 2023-01-02 23:22:12 wer_bpe:260] 
    
[NeMo I 2023-01-02 23:22:12 wer_bpe:261] reference:rangierfahrt von quebec fünf acht ins gleis oscar sieben drei antworten
[NeMo I 2023-01-02 23:2